<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://bokeh.pydata.org/">     
           <img 
               src="assets/bokeh-transparent.png" 
               style="width:50px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Bokeh 教程</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>A1. 模型和原语</h2></div>

# 概述

虽然被叫做一个名字，但 Bokeh 却是由两个单独的库组成。

第一个是JavaScript库，BokehJS，在浏览器中运行。这个库负责所有的呈现和用户交互。它的输入是一系列陈述性JSON对象组成的“场景图”。这个场景图中的对象描述了BokehJS要处理的一切：要呈现的plots和widgets，怎么排列，有什么工具、渲染器、轴，等等。这些JSON对象在浏览器中被转换成JavaScript对象。

第二个是Python（或其他语言）写的用来生成上述JSON的一个库。Python Bokeh库中定义了一组底层 `Model` 子类，很好地镜像了浏览器中使用的BokehJS模型。大多数模型非常简单，通常由几个属性（attribute）组成，没有方法（method）。模型属性可以在创建模型时设置，也可以通过模型对象来设置其值：

#### 属性可以在模型对象初始化的时候设置
```python
glyph = Rect(x="x", y="y2", w=10, h=20, line_color=None)
```

#### 属性可以通过赋值给模型对象的成员来设置
```python
glyph.fill_alpha = 0.5
glyph.fill_color = "navy"
```

这些属性设置方法对所有Bokeh模型都适用。

为了 bokehJS 和 Bokeh Python绑定之间的序列化，Bokeh模型最终被收集到 Bokeh `Document` 里。举个例子，下面的图片展示了一个简单图表的示例文档：

![Document Structure](assets/document.svg)

这样的文档可以使用高级的 `bokeh.plotting` API 创建，自动合理地组织适当的模型如轴线、网格和工具栏。也可以使用低级的 `bokeh.models` API “手动”组装所有的部件。使用 `bokeh.models` 接口提供了对Bokeh plots和widgets如何布置安排和配置的完全控制。然而，它不提供如何组装模型的帮助，完全由开发者“手动”建立场景。

更多Bokeh模型的细节信息，请参考 [参考手册](http://bokeh.pydata.org/en/latest/docs/reference.html).

# 示例演练

让我们尝试使用 `bokeh.models` 接口重现这个纽约时报的交互式图表 [Usain Bolt vs. 116 years of Olympic sprinters](http://www.nytimes.com/interactive/2012/08/05/sports/olympics/the-100-meter-dash-one-race-every-medalist-ever.html)。

我们需要做的第一件事就是获取数据。此图表的数据可在 ``bokeh.sampledata`` 模块中找到，作为一个Pandas DataFrame数据。您可以看看前十行数据：

In [1]:
from bokeh.sampledata.sprint import sprint
sprint[:10]

,Name,Country,Medal,Time,Year
0,Usain Bolt,JAM,GOLD,9.63,2012
1,Yohan Blake,JAM,SILVER,9.75,2012
2,Justin Gatlin,USA,BRONZE,9.79,2012
3,Usain Bolt,JAM,GOLD,9.69,2008
4,Richard Thompson,TRI,SILVER,9.89,2008
5,Walter Dix,USA,BRONZE,9.91,2008
6,Justin Gatlin,USA,GOLD,9.85,2004
7,Francis Obikwelu,POR,SILVER,9.86,2004
8,Maurice Greene,USA,BRONZE,9.87,2004
9,Maurice Greene,USA,GOLD,9.87,2000


下一步，我们导入一些组合图表所需的Bokeh模型。不用太多，我们从 ``Plot`` 开始，还有要显示的标记符（``Circle`` 和 ``Text``），以及保存数据的 ``ColumnDataSource`` 和设置图表界限的 range 对象。

In [2]:
from bokeh.io import output_notebook, show
from bokeh.models.glyphs import Circle, Text
from bokeh.models import ColumnDataSource, Range1d, DataRange1d, Plot

In [3]:
output_notebook()

Loading BokehJS ...

## Setting up Data

接下来，我们需要在列数据源中设置我们所有想要的列。在这里，我们添加一些额外的列如 `MetersBack` 和 `SelectedName`（后面我们做 `HoverTool` 的时候会用到）。

In [4]:
abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

fill_color = { "gold": "#efcf6d", "silver": "#cccccc", "bronze": "#c59e8a" }
line_color = { "gold": "#c8a850", "silver": "#b0b0b1", "bronze": "#98715d" }

def selected_name(name, medal, year):
    return name if medal == "gold" and year in [1988, 1968, 1936, 1896] else ""

t0 = sprint.Time[0]

sprint["Abbrev"]       = sprint.Country
sprint["Country"]      = sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])
sprint["Medal"]        = sprint.Medal.map(lambda medal: medal.lower())
sprint["Speed"]        = 100.0/sprint.Time
sprint["MetersBack"]   = 100.0*(1.0 - t0/sprint.Time)
sprint["MedalFill"]    = sprint.Medal.map(lambda medal: fill_color[medal])
sprint["MedalLine"]    = sprint.Medal.map(lambda medal: line_color[medal])
sprint["SelectedName"] = sprint[["Name", "Medal", "Year"]].apply(tuple, axis=1).map(lambda args: selected_name(*args))

source = ColumnDataSource(sprint)

## 逐步修建图表

让我们逐步修建我们的图表，时不时停下来检查下输出，看看修得怎么样了。
Let's build up our plot in stages, stopping to check the output along the way to see how things look.

当我们开始时，请注意 `Plot`, `Chart`, 和 `Figure`都有的三个调用：

* `p.add_glyph`
* `p.add_tools`
* `p.add_layout`

这些便利调用方法帮助我们以正确的方式添加模型到 `Plot` 对象中。

### 只有标记符的基本图表

首先，我们创建一个只有标题和一些基本样式的 `Plot`，并为实际的比赛数据添加一些 `Circle` 标记符。为了手动配置标记符，我们首先创建一个glyph对象（例如，`Text`或`Circle`），设置我们想要的视觉样式以及坐标数据。然后，我们调用 `plot.add_glyph` 添加标记符和相应的数据。

In [5]:
plot_options = dict(plot_width=800, plot_height=480, toolbar_location=None, outline_line_color=None)

In [6]:
medal_glyph = Circle(x="MetersBack", y="Year", size=10, fill_color="MedalFill", 
                     line_color="MedalLine", fill_alpha=0.5)

athlete_glyph = Text(x="MetersBack", y="Year", x_offset=10, text="SelectedName",
                     text_align="left", text_baseline="middle", text_font_size="9pt")

no_olympics_glyph = Text(x=7.5, y=1942, text=["No Olympics in 1940 or 1944"],
                         text_align="center", text_baseline="middle",
                         text_font_size="9pt", text_font_style="italic", text_color="silver")

In [7]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

GlyphRenderer(id='9539a4a4-b166-4428-85da-4ec4d8ec9525', ...)

In [8]:
show(plot)

### 加入轴线和网格线

接下来，我们添加我们想看到的 `Axis` 和 `Grids` 模型。因为我们想要对外观进行更多的控制，所以我们选择一个有特殊刻度的轴模型（这里选择`singleintervalticker`）。然后，我们使用 `plot.add_layout` 方法把它们添加到图表。

In [9]:
from bokeh.models import Grid, LinearAxis, SingleIntervalTicker

In [10]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

GlyphRenderer(id='0fecf8b7-44e7-4c31-8477-a249cc03473b', ...)

In [11]:
xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [12]:
show(plot)

### 添加一个悬停工具

最后，我们添加一个悬停工具来显示我们在列数据源中另外添加的列数据。为了对外观样式进行更多的控制，我们使用模板语法来设置工具提示。工具可以使用 `plot.add_tools` 方法添加。

In [13]:
from bokeh.models import HoverTool

In [14]:
tooltips = """
<div>
    <span style="font-size: 15px;">@Name</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@Abbrev)</span>
</div>
<div>
    <span style="font-size: 17px; font-weight: bold;">@Time{0.00}</span>&nbsp;
    <span style="font-size: 10px; color: #666;">@Year</span>
</div>
<div style="font-size: 11px; color: #666;">@{MetersBack}{0.00} meters behind</div>
"""

In [15]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
medal = plot.add_glyph(source, medal_glyph)  # we need this renderer to configure the hover tool
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [16]:
hover = HoverTool(tooltips=tooltips, renderers=[medal])
plot.add_tools(hover)

In [17]:
show(plot)

# 作业

# 定制用户模型

使用用户自定义模型来扩充Bokeh内置模型库是可行的。某些情况下，该功能是很有价值的用：
* 自定义现有的Bokeh模型的行为
* 包装和连接其他JS库到Bokeh

有了这项功能，在无需建立一个完整的Bokeh发展环境的情况下，高级用户可以轻松地尝试新的功能或技术。

本节给出了一个用JavaScript实现自定义模型的基本轮廓，继承一个现有的BokehJS模型。详情请参考用户手册中 [扩展 Bokeh](http://bokeh.pydata.org/en/latest/docs/user_guide/extensions.html) 章节。

### 实现 JavaScript 模型

In [18]:
JS_CODE = """
import {div, empty} from "core/dom"

# The "core/properties" module has all the property types
import * as p from "core/properties"

# We will subclass in JavaScript from the same class that was subclassed
# from in Python
import {LayoutDOM, LayoutDOMView} from "models/layouts/layout_dom"

# This model will actually need to render things, so we must provide
# view. The LayoutDOM model has a view already, so we will start with that
export class CustomView extends LayoutDOMView

  initialize: (options) ->
    super(options)

    @render()

    # Set Backbone listener so that when the Bokeh slider has a change
    # event, we can process the new data
    @connect(@model.slider.change, () => @render())

  render: () ->
    # Backbone Views create <div> elements by default, accessible as @el.
    # Many Bokeh views ignore this default <div>, and instead do things
    # like draw to the HTML canvas. In this case though, we change the
    # contents of the <div>, based on the current slider value.
    empty(@el)
    @el.appendChild(div({
      style: {
        color: '#686d8e'
        'background-color': '#2a3153'
      }
    }, "#{@model.text}: #{@model.slider.value}"))

export class Custom extends LayoutDOM

  # If there is an associated view, this is boilerplate.
  default_view: CustomView

  # The ``type`` class attribute should generally match exactly the name
  # of the corresponding Python class.
  type: "Custom"

  # The @define block adds corresponding "properties" to the JS model. These
  # should basically line up 1-1 with the Python model class. Most property
  # types have counterparts, e.g. bokeh.core.properties.String will be
  # p.String in the JS implementation. Where the JS type system is not yet
  # as rich, you can use p.Any as a "wildcard" property type.
  @define {
    text:   [ p.String ]
    slider: [ p.Any    ]
  }
"""

### 定义 Python 模型

把 JavaScript 实现附加到对应的 Python Bokeh 模型：

In [19]:
from bokeh.core.properties import String, Instance
from bokeh.models import LayoutDOM, Slider

class Custom(LayoutDOM):

    __implementation__ = JS_CODE

    text = String(default="Custom text")

    slider = Instance(Slider)

### 使用 Python 模型

然后，新定义的模型就可以像内置的Bokeh模型一样使用了。

In [20]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import Slider

slider = Slider(start=0, end=10, step=0.1, value=0, title="value")

custom = Custom(text="Special Slider Display", slider=slider)

layout = column(slider, custom)

# Currently a bug preventing custom extensions working in notebooks
#show(layout)